# Test the VLM endpoint

In [ ]:
import base64
import cv2
import numpy as np
import pyheif
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from pathlib import Path
import io
from PIL import Image as PIL_Image
from IPython.display import display, Image as IPython_Image

def display_image(image_path: str):
    if Path(image_path).suffix.lower() == '.heic':
        try:
            heif_file = pyheif.read(image_path)
            image = PIL_Image.frombytes(
                heif_file.mode, 
                heif_file.size, 
                heif_file.data,
                "raw",
            )
            buffer = io.BytesIO()
            image.save(buffer, format="PNG")
            display(IPython_Image(data=buffer.getvalue(), width=300))
        except Exception as e:
            print(f"Could not process HEIC file: {e}")
    else:
        display(IPython_Image(filename=image_path, width=300))


def get_all_files(directory: str) -> list[str]:
    IMAGE_EXTENSIONS = {
        '.jpg', '.jpeg', '.heic',
    }
    return [str(p) for p in Path(directory).rglob('*') if p.is_file() and p.suffix.lower() in IMAGE_EXTENSIONS]

# Function to encode an image
def encode_image(image_path: str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to encode and resize an image
def encode_and_resize(image_path: str, max_size: int = 640):
    if image_path.endswith(".HEIC"):
        img = read_heic_to_numpy(image_path)
    else:
        img = cv2.imread(image_path)
    scale_factor = max_size / max(img.shape)
    img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    _, im_arr = cv2.imencode('.jpg', img)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    return base64.b64encode(im_bytes).decode("utf-8")

# Function to read .HEIC image format
def read_heic_to_numpy(file_path: str):
    heif_file = pyheif.read(file_path)
    data = heif_file.data
    if heif_file.mode == "RGB":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 3)
    elif heif_file.mode == "RGBA":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 4)
    else:
        raise ValueError("Unsupported HEIC color mode")
    return numpy_array


endpoint = "https://oai-aip-cv-ont-sdc.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

## Ask question with a photo as context

In [ ]:
def get_response(base64_image: str):
    """Each team can customize this function as they want to get the best results for their use case.

    Parameters
    ----------
    base64_image : str
        Base64 encoded image

    Returns
    -------
    str
        Text response from the model according to the prompt
    """
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a sidewalk inspector for the municipality. Your job is to inspect images taken of sidewalk surfaces and assess their quality and maintenance needs.",
            },
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": "Describe the quality of the sidewalk in this image." },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "low", # reduces token usage
                        },
                    },
                ],
            }
        ],
        max_tokens=4096,
        temperature=1.0,
        top_p=1.0,
        model=deployment
    )
    return response.choices[0].message.content

In [ ]:
images_to_check = get_all_files("notebooks/local_data/TestDataHackatonVLM/")

for image_path in images_to_check:
    display_image(image_path)
    base64_image = encode_image(image_path)
    print(get_response(base64_image))